# Get data from NIRD archive via minio s3-compatible
### Import python packages

In [1]:
import os
import requests
import s3fs
import xarray as xr

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import cftime

### Access to data via minio

In [2]:
nird_storage = s3fs.S3FileSystem(  
            anon=True,
            client_kwargs={"endpoint_url": "https://galaxy-minio.uiogeo-apps.sigma2.no/" },
        )

### List folders/files from bucket

In [3]:
bucket = 'oracle-arm-1x80'
bucket_content = nird_storage.ls(bucket)
print(bucket_content)

['oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-01.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-02.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-03.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-04.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-05.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-06.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-07.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-08.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-09.nc', 'oracle-a

In [4]:
s3path = 's3://' + bucket + '/*.cam.h0.*.nc'

In [5]:
remote_files = nird_storage.glob(s3path)
print(remote_files)

['oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-01.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-02.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-03.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-04.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-05.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-06.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-07.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-08.nc', 'oracle-arm-1x80/1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19_f19_mg17-20210525.cam.h0.0001-09.nc', 'oracle-a

In [6]:
fileset = [nird_storage.open(file) for file in remote_files]

#### Open remote files

In [7]:
data = xr.open_mfdataset(fileset)

In [8]:
data

<xarray.Dataset>
Dimensions:           (ilev: 33, lat: 96, lev: 32, lon: 144, nbnd: 2, time: 24, zlon: 1)
Coordinates:
  * lat               (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * zlon              (zlon) float64 0.0
  * lon               (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lev               (lev) float64 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * ilev              (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time              (time) object 0001-02-01 00:00:00 ... 0003-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/414)
    zlon_bnds         (time, zlon, nbnd) float64 dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    gw                (time, lat) float64 dask.array<chunksize=(1, 96), meta=np.ndarray>
    hyam              (time, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    hybm              (time, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    P0                (time) float64 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    hyai              (time, ilev) float64 dask.array<chunksize=(1, 33), meta=np.ndarray>
    ...                ...
    mmr_OM            (time, lev, lat, lon) float32 dask.array<chunksize=(1, 32, 96, 144), meta=np.ndarray>
    mmr_SALT          (time, lev, lat, lon) float32 dask.array<chunksize=(1, 32, 96, 144), meta=np.ndarray>
    mmr_SULFATE       (time, lev, lat, lon) float32 dask.array<chunksize=(1, 32, 96, 144), meta=np.ndarray>
    monoterp          (time, lev, lat, lon) float32 dask.array<chunksize=(1, 32, 96, 144), meta=np.ndarray>
    monoterp_SRF      (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
    odms              (time, lat, lon) float32 dask.array<chunksize=(1, 96, 144), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              1x80-aarm64-noresm-gnu-mpich-container-NF2000climo-f19...
    logname:           opc
    host:              
    initial_file:      /opt/esm/inputdata/atm/cam/inic/fv/cami-mam3_0000-01-0...
    topography_file:   /opt/esm/inputdata/atm/cam/topo/fv_1.9x2.5_nc3000_Nsw0...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [ ]:
fig = plt.figure(1, figsize=[20,10])
tsel = 0 # select first date e.g. 1st January 251
# Fix extent
minval = 240
maxval = 300

for tsel in range(len(data.TREFHT.time)):

    # Plot 1 for Northern Hemisphere subplot argument (nrows, ncols, nplot)
    # here 1 row, 2 columns and 1st plot
    ax = plt.subplot(1, 1, 1, projection=ccrs.Orthographic(0, 50))

    # Plot 4xCO2 experiment
    map = data['TREFHT'].isel(time=tsel).plot(ax=ax, vmin=minval, vmax=maxval, 
                                        transform=ccrs.PlateCarree(), 
                                        cmap='coolwarm', 
                                        add_colorbar=False)
    ax.set_title( data.TREFHT.attrs['long_name'] + '\n' + data.time.values[tsel].strftime("%B Year %Y"), fontsize=15 )
    ax.coastlines()
    ax.gridlines()


    # Title for both plots
    fig.suptitle( "Oracle Cloud Infractructure 1x80 ARM processors" , fontsize=20 )


    cb_ax = fig.add_axes([0.325, 0.05, 0.4, 0.04])

    cbar = plt.colorbar(map, cax=cb_ax, extend='both', orientation='horizontal', fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=25)
    cbar.ax.set_ylabel('K', fontsize=25)
    plt.savefig('TREFHT' + data.time.values[tsel].strftime("%Y%m%d").strip() + '.png')
    plt.clf()